# Perform Balancing of Dataset

From the previously unbalanced dataset, we:
- Remove all real (label 0) videos
- Add real counterparts to deepfake videos from each dataset
- Reorganize, rename and compress these new set of real videos

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path  

main_df = pd.read_csv('D:/collated/labels_with_original_path.csv')
main_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path
0,0000.mp4,0505.mp4,1,lipsync-voiceclone,SelfGenerated,c40,decowa_3,wav2lip-elevenlabs,CSA seminar emma watson,D:/datasets/augmented\EW_for_benchmark_fake_de...
1,0001.mp4,2075.mp4,1,faceswap,FaceForensics++,c23,-,-,-,D:\datasets\ff-videos\manipulated_sequences\Fa...
2,0002.mp4,-,1,faceswap,CelebDF,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id7_id...
3,0003.mp4,0570.mp4,1,faceswap,DFDC,c40,-,-,-,D:\datasets\DFDC\1\ciyiyjdxsj.mp4
4,0004.mp4,-,1,lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
...,...,...,...,...,...,...,...,...,...,...
2349,2349.mp4,-,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
2350,2350.mp4,-,1,faceswap,CelebDF,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id50_i...
2351,2351.mp4,-,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
2352,2352.mp4,-,1,lipsync,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-RealAud...


In [2]:
remove_df = main_df[((main_df['label'] == 0) & (main_df['source'] != 'SelfGenerated')) | (main_df['source'] == 'FaceForensics++')]
main_df = main_df[((main_df['label'] == 1) | (main_df['source'] == 'SelfGenerated')) & (main_df['source'] != 'FaceForensics++')]

In [3]:
main_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path
0,0000.mp4,0505.mp4,1,lipsync-voiceclone,SelfGenerated,c40,decowa_3,wav2lip-elevenlabs,CSA seminar emma watson,D:/datasets/augmented\EW_for_benchmark_fake_de...
2,0002.mp4,-,1,faceswap,CelebDF,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id7_id...
3,0003.mp4,0570.mp4,1,faceswap,DFDC,c40,-,-,-,D:\datasets\DFDC\1\ciyiyjdxsj.mp4
4,0004.mp4,-,1,lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
5,0005.mp4,0458.mp4,1,lipsync-voiceclone,SelfGenerated,c40,ncs_3,wav2lip-elevenlabs,CSA anniversary lee hsien loong,D:/datasets/augmented\pmlee_fake_1_ncs_3.mp4
...,...,...,...,...,...,...,...,...,...,...
2349,2349.mp4,-,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
2350,2350.mp4,-,1,faceswap,CelebDF,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id50_i...
2351,2351.mp4,-,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...
2352,2352.mp4,-,1,lipsync,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-RealAud...


In [4]:
new_df = pd.DataFrame(columns=['name', 'label', 'technique', 'source', 'target', 'compression', 'augmentation', 'tool', 'content', 'original_path'])
running_count = 5000

for index, row in main_df.iterrows():
    if row['augmentation'] != '-' or row['compression'] != 'original':
        continue
    if row['source'] == 'FakeAVCeleb':
        filename = Path(row['original_path']).name
        target_path = Path(row['original_path'].replace('FakeVideo-FakeAudio', 'RealVideo-RealAudio').replace('FakeVideo-RealAudio', 'RealVideo-RealAudio')).parent
        try:
            target = filename.split('_')[1]
        except:
            continue
        if 'id' in target:
            target_path = target_path.parent / target
        target_path_file = [os.path.join(target_path, f) for f in os.listdir(target_path) if '.mp4' in f][0]
        new_row = row
        new_row['name'] = f'{running_count}.mp4'
        running_count += 1
        new_row['original_path'] = target_path_file
        new_row['label'] = 0
        main_df.at[index, 'target'] = new_row['name']
        new_df = new_df._append(new_row, ignore_index=True)
            
    

In [5]:

for index, row in main_df.iterrows():
    if row['augmentation'] != '-' or row['compression'] != 'original':
        continue
    if row['source'] == 'CelebDF':
        filename = Path(row['original_path']).name
        target_path = Path(row['original_path'].replace('Celeb-synthesis', 'Celeb-real')).parent
        try:
            target = filename.split('_')[0] + '_' + filename.split('_')[2]
        except:
            print(filename)
            continue
        target_path_file = target_path / target
        new_row = row
        new_row['name'] = f'{running_count}.mp4'
        running_count += 1
        new_row['original_path'] = target_path_file
        new_row['label'] = 0
        main_df.at[index, 'target'] = new_row['name']
        new_df = new_df._append(new_row, ignore_index=True)

In [6]:
new_df

,name,label,technique,source,target,compression,augmentation,tool,content,original_path
0,5000.mp4,0,lipsync-voiceclone,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
1,5001.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
2,5002.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
3,5003.mp4,0,faceswap-lipsync-voiceclone,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
4,5004.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
...,...,...,...,...,...,...,...,...,...,...
392,5392.mp4,0,faceswap,CelebDF,-,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-real\id3_0002.mp4
393,5393.mp4,0,faceswap,CelebDF,-,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-real\id9_0005.mp4
394,5394.mp4,0,faceswap,CelebDF,-,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-real\id27_0007.mp4
395,5395.mp4,0,faceswap,CelebDF,-,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-real\id23_0003.mp4


In [7]:
existing_original_dfd = main_df[(main_df['source'] == 'DFD') & (main_df['label'] == 1) & (main_df['compression'] == 'original')]['original_path']
count = 0
path_to_dfd = Path('D:/datasets/ff-videos/original_sequences/actors/c23/videos')
    # get list of video paths in folder without visiting subdirectories
video_list =  [os.path.join(path_to_dfd, f) for f in os.listdir(path_to_dfd) if os.path.isfile(os.path.join(path_to_dfd, f))]
for video in video_list:
    if video not in existing_original_dfd:
        new_row = {'name': f'{running_count}.mp4', 'label': 0, 
            'technique': 'faceswap', 'source': 'DFD', 'target': '-', 
            'compression': 'original', 'augmentation': '-', 'tool': '-', 'content': '-', 'original_path': video}
        count += 1
        running_count += 1
        new_df = new_df._append(new_row, ignore_index = True)
        if count == 90:
            break
        


In [8]:
import random
count = 0
base_paths = [ "manipulated_sequences/Deepfakes/c23/videos", \
                            "manipulated_sequences/Face2Face/c23/videos", "manipulated_sequences/FaceSwap/c23/videos", \
                            "manipulated_sequences/NeuralTextures/c23/videos"]

paths_to_FF = [Path(f'D:/datasets/ff-videos/{i}') for i in base_paths]

source_to_name = {}

for path in paths_to_FF:
    video_list = random.sample([os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))], 50)
    for video in video_list:
        add_source = True
        filename = Path(video).name
        source = filename.split('_')[0] + '.mp4'
        path_to_source = f'D:/datasets/ff-videos/original_sequences/youtube/c23/videos/{source}'
        if source not in source_to_name:
            source_to_name[source] = f'{running_count}.mp4'
            target_map = f'{running_count}.mp4'
        else:
            target_map = source_to_name[source]
            add_source = False
        
        if add_source:
            new_row = {'name': f'{running_count}.mp4', 'label': 0, 
                    'technique': '-', 'source': 'FaceForensics++', 'target': '-', 'compression': 'original', 'augmentation': '-', 
                    'tool': '-', 'content': '-', 'original_path': path_to_source}
            running_count += 1
            new_df = new_df._append(new_row, ignore_index = True)
        new_row_2 = {'name': f'{running_count}.mp4', 'label': 1, 
                'technique': 'faceswap' if 'Deepfakes' in video or 'FaceSwap' in video else 'facemanip', 
                'source': 'FaceForensics++', 'target': target_map, 'compression': 'original', 'augmentation': '-', 
                'tool': '-', 'content': '-', 'original_path': video}
        running_count += 1
        new_df = new_df._append(new_row_2, ignore_index = True)

In [9]:
path_to_DFDC = Path('D:/datasets/DFDC')
label_file = Path('D:/datasets/DFDC/labels.csv')
label_df = pd.read_csv(label_file)

video_list = []        
original_df = label_df[label_df['label'] == 0].sample(200)
for file in original_df['filename']:
    for path in ['0', '1', '2', '3', '4']:
        file_path = path_to_DFDC / path / file
        if os.path.isfile(file_path):
            new_row = {'name': f'{running_count}.mp4', 'label': 0, 
                'technique': '-', 'source': 'DFDC', 'target': '-', 'compression': 'original', 'augmentation': '-', 
                'tool': '-', 'content': '-', 'original_path': file_path}
            running_count += 1
            new_df = new_df._append(new_row, ignore_index = True)
            break

In [10]:
new_df

,name,label,technique,source,target,compression,augmentation,tool,content,original_path
0,5000.mp4,0,lipsync-voiceclone,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
1,5001.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
2,5002.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
3,5003.mp4,0,faceswap-lipsync-voiceclone,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
4,5004.mp4,0,faceswap,FakeAVCeleb,-,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
...,...,...,...,...,...,...,...,...,...,...
1068,6068.mp4,0,-,DFDC,-,original,-,-,-,D:\datasets\DFDC\3\oswxywxbgs.mp4
1069,6069.mp4,0,-,DFDC,-,original,-,-,-,D:\datasets\DFDC\3\vnorypglil.mp4
1070,6070.mp4,0,-,DFDC,-,original,-,-,-,D:\datasets\DFDC\3\hlorwkxuum.mp4
1071,6071.mp4,0,-,DFDC,-,original,-,-,-,D:\datasets\DFDC\0\dhafaeocni.mp4


In [11]:
for index, row in remove_df.iterrows():
    path_to_remove = Path(f"D:\collated\{row['name']}")
    os.remove(path_to_remove)

In [12]:
compressed_df = new_df.copy()
compressed_df['target'] = compressed_df['name']
indices = random.sample(range(len(compressed_df)), len(compressed_df)//2)
compressed_df['compression'] = 'c23'
compressed_df.loc[indices,'compression'] = 'c40'

for index, row in compressed_df.iterrows():
    compressed_df.at[index, 'name'] = f'{running_count}.mp4'
    running_count += 1

In [13]:
compressed_df

,name,label,technique,source,target,compression,augmentation,tool,content,original_path
0,6073.mp4,0,lipsync-voiceclone,FakeAVCeleb,5000.mp4,c40,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
1,6074.mp4,0,faceswap,FakeAVCeleb,5001.mp4,c23,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
2,6075.mp4,0,faceswap,FakeAVCeleb,5002.mp4,c40,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
3,6076.mp4,0,faceswap-lipsync-voiceclone,FakeAVCeleb,5003.mp4,c40,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
4,6077.mp4,0,faceswap,FakeAVCeleb,5004.mp4,c40,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...
...,...,...,...,...,...,...,...,...,...,...
1068,7141.mp4,0,-,DFDC,6068.mp4,c40,-,-,-,D:\datasets\DFDC\3\oswxywxbgs.mp4
1069,7142.mp4,0,-,DFDC,6069.mp4,c23,-,-,-,D:\datasets\DFDC\3\vnorypglil.mp4
1070,7143.mp4,0,-,DFDC,6070.mp4,c23,-,-,-,D:\datasets\DFDC\3\hlorwkxuum.mp4
1071,7144.mp4,0,-,DFDC,6071.mp4,c40,-,-,-,D:\datasets\DFDC\0\dhafaeocni.mp4


In [14]:
new_df['from'] = 'new'
compressed_df['from'] = 'new'
main_df['from'] = 'old'
combined_df = pd.concat([main_df, new_df, compressed_df], ignore_index=True)

C:\Users\silas\AppData\Local\Temp\ipykernel_6936\1984715274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['from'] = 'old'


In [15]:
combined_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path,from
0,0000.mp4,0505.mp4,1,lipsync-voiceclone,SelfGenerated,c40,decowa_3,wav2lip-elevenlabs,CSA seminar emma watson,D:/datasets/augmented\EW_for_benchmark_fake_de...,old
1,0002.mp4,5197.mp4,1,faceswap,CelebDF,original,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id7_id...,old
2,0003.mp4,0570.mp4,1,faceswap,DFDC,c40,-,-,-,D:\datasets\DFDC\1\ciyiyjdxsj.mp4,old
3,0004.mp4,5000.mp4,1,lipsync-voiceclone,FakeAVCeleb,original,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-FakeAud...,old
4,0005.mp4,0458.mp4,1,lipsync-voiceclone,SelfGenerated,c40,ncs_3,wav2lip-elevenlabs,CSA anniversary lee hsien loong,D:/datasets/augmented\pmlee_fake_1_ncs_3.mp4,old
...,...,...,...,...,...,...,...,...,...,...,...
3781,7141.mp4,6068.mp4,0,-,DFDC,c40,-,-,-,D:\datasets\DFDC\3\oswxywxbgs.mp4,new
3782,7142.mp4,6069.mp4,0,-,DFDC,c23,-,-,-,D:\datasets\DFDC\3\vnorypglil.mp4,new
3783,7143.mp4,6070.mp4,0,-,DFDC,c23,-,-,-,D:\datasets\DFDC\3\hlorwkxuum.mp4,new
3784,7144.mp4,6071.mp4,0,-,DFDC,c40,-,-,-,D:\datasets\DFDC\0\dhafaeocni.mp4,new


In [16]:
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df['old_name'] = combined_df['name']
combined_df['name'] = combined_df.index.to_series().apply(lambda x: f'{x:04d}.mp4')

original_to_new = {}
for index, row in combined_df.iterrows():
    original_to_new[row['old_name']] = row['name']

for index, row in combined_df.iterrows():
    if row['target'] != '-':
        combined_df.at[index, 'target'] = original_to_new[row['target']]

In [17]:
combined_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path,from,old_name
0,0000.mp4,2174.mp4,1,faceswap,CelebDF,original,ncs_3,-,-,D:\datasets\augmented\id51_id53_0004_ncs_3.mp4,old,1645.mp4
1,0001.mp4,2228.mp4,0,faceswap,FakeAVCeleb,c23,-,-,-,D:\datasets\FakeAVCeleb_v1.2\RealVideo-RealAud...,new,6104.mp4
2,0002.mp4,0002.mp4,1,faceswap,DFD,c40,-,-,-,D:\datasets\ff-videos\manipulated_sequences\De...,old,1412.mp4
3,0003.mp4,1026.mp4,0,faceswap,CelebDF,c23,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-real\id48_0001.mp4,new,6462.mp4
4,0004.mp4,1903.mp4,1,faceswap,FakeAVCeleb,c23,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-RealAud...,old,0836.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...
3781,3781.mp4,3598.mp4,1,faceswap,FaceForensics++,original,-,-,-,D:\datasets\ff-videos\manipulated_sequences\De...,new,5492.mp4
3782,3782.mp4,2543.mp4,1,faceswap,CelebDF,c40,gra_3,-,-,D:\datasets\augmented\id51_id53_0004_gra_3.mp4,old,0255.mp4
3783,3783.mp4,-,0,-,FaceForensics++,original,-,-,-,D:/datasets/ff-videos/original_sequences/youtu...,new,5623.mp4
3784,3784.mp4,-,1,faceswap,DFD,original,-,-,-,D:\datasets\ff-videos\manipulated_sequences\De...,old,1203.mp4


In [18]:
output_dir = Path('D:/collated/new')
old_dir = Path('D:/collated')
os.makedirs(output_dir, exist_ok=True)
ffmpeg_compress = 'ffmpeg -i "{input_video_file}" -vcodec libx264 -y -crf {rate_factor} {output_video_file}'

for _, row in combined_df.iterrows():
    output_video_file = output_dir / row['name']
    if row['from'] == 'old':
        shutil.move(old_dir / row['old_name'], output_video_file)
        continue
    if row['compression'] == 'c23':
        os.system(ffmpeg_compress.format(input_video_file = row['original_path'], output_video_file = output_video_file, rate_factor = 23))
    elif row['compression'] == 'c40':
        os.system(ffmpeg_compress.format(input_video_file = row['original_path'], output_video_file = output_video_file, rate_factor = 40))
    else:
        shutil.copy(row['original_path'], output_video_file)

In [19]:
output_dir = Path('D:/collated')
combined_df.to_csv(os.path.join(output_dir, 'labels.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content'])
    
combined_df.to_csv(os.path.join(output_dir, 'labels_with_original_path.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content', 'original_path'])
